In [2]:
"""
Блок 3. Оптимизация с помощью физических джойнов
"""

import libs.db_connection

# Hash join
sql1 = """explain (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
and a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Gather  (cost=1001.25..15646.03 rows=1 width=103) (actual time=208.829..210.628 rows=0 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Hash Join  (cost=1.25..14645.93 rows=1 width=103) (actual time=203.628..203.631 rows=0 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '        Hash Cond: ((a.user_id = b.user_id) AND ((a.user_name)::text = (b.user_name)::text))')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Parallel Seq Scan on block51 a  (cost=0.00..11519.67 rows=416667 width=27) (actual time=1.947..151.574 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Hash  (cost=1.10..1.10 rows=10 width=76) (actual time=0.037..0.038 rows=10 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '              Buckets: 1024  Batches: 1  Memory Usage: 9kB')]),
 Real

In [4]:
# Nested Loop join
sql1 = """explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
or a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Nested Loop  (cost=0.00..192354.12 rows=215 width=103) (actual time=17.206..1245.846 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  Join Filter: ((a.user_id = b.user_id) OR ((a.user_name)::text = (b.user_name)::text))')]),
 RealDictRow([('QUERY PLAN', '  Rows Removed by Join Filter: 9999794')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Seq Scan on block51 a  (cost=0.00..17353.00 rows=1000000 width=27) (actual time=0.065..75.381 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Materialize  (cost=0.00..1.15 rows=10 width=76) (actual time=0.000..0.000 rows=10 loops=1000000)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Seq Scan on block53 b  (cost=0.00..1.10 rows=10 width=76) (actual time=0.004..0.009 rows=10 loops=1)')]),
 RealDictRow([('QUERY PLAN', 'Planning Time: 0.201 ms')]),
 RealDictRow([('QUERY PLAN', 'JIT:')]),
 RealDictRow([('QUERY PLAN', '  Functions: 6')])

In [5]:
# Оптимизация 
sql1 = """
explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id

union

select * 
from block51 a 
inner join block53 b  
on a.user_name = b.user_name
"""

libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'HashAggregate  (cost=27197.25..27199.40 rows=215 width=152) (actual time=99.979..100.743 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  Group Key: a.event_id, a.user_id, a.user_name, a.category, a.type, b.user_id, b.user_name, b.type, b.register_dt')]),
 RealDictRow([('QUERY PLAN', '  Batches: 1  Memory Usage: 80kB')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Gather  (cost=1001.23..27192.41 rows=215 width=152) (actual time=1.674..99.427 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '        Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Parallel Append  (cost=1.23..26170.91 rows=215 width=152) (actual time=2.735..96.519 rows=69 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Hash Join  (cost=1.23..13083.84 rows=45 width=103) (actual time=0.670..41.972 rows=35 loops=3)')]),
 Real

In [6]:
# Создаем индекс
sql1 = "create index block11_tst on block51 (user_id, user_name)"

            
libs.db_connection.execute(sql1)

In [7]:
# Создаем индекс
sql1 = "create index block53_tst on block53 (user_id, user_name)"

            
libs.db_connection.execute(sql1)

In [8]:
# Nested Loop join
sql1 = """explain  (analyze) 
select * 
from block51 a 
inner join block53 b  
on a.user_id = b.user_id 
or a.user_name = b.user_name"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Nested Loop  (cost=9046.14..91274.11 rows=215 width=103) (actual time=40.997..252.586 rows=206 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Seq Scan on block53 b  (cost=0.00..1.10 rows=10 width=76) (actual time=0.004..0.016 rows=10 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Bitmap Heap Scan on block51 a  (cost=9046.14..9127.09 rows=21 width=27) (actual time=25.142..25.242 rows=21 loops=10)')]),
 RealDictRow([('QUERY PLAN',
               '        Recheck Cond: ((user_id = b.user_id) OR ((user_name)::text = (b.user_name)::text))')]),
 RealDictRow([('QUERY PLAN', '        Heap Blocks: exact=206')]),
 RealDictRow([('QUERY PLAN',
               '        ->  BitmapOr  (cost=9046.14..9046.14 rows=21 width=0) (actual time=25.110..25.110 rows=0 loops=10)')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Bitmap Index Scan on block11_tst  (cost=0.00..4.51 rows=11 width=0) (actual time=0.012..